In [65]:
# Irreducible polynomial for AES: x^8 + x^4 + x^3 + x + 1
MOD_POLY = 0x11B

# Multiply two numbers in GF(2^8)
def gf_mul(a: int, b: int) -> int:
    result = 0
    for _ in range(8):
        if b & 1:
            result ^= a
        hi_bit_set = a & 0x80
        a <<= 1
        a &= 0xFF  # keep it 8-bit
        if hi_bit_set:
            a ^= 0x1B  # reduction by x^8 + x^4 + x^3 + x + 1
        b >>= 1
    return result

# Find multiplicative inverse in GF(2^8)
def gf_inv(a: int) -> int:
    if a == 0:
        return 0  # 0 has no inverse
    
    result = 1
    base = a
    for _ in range(254):  # a^(254) = a^(-1) in GF(2^8)
        result = gf_mul(result, base)
    return result




In [73]:
val=0x3d
inv=gf_inv(val)
print(inv)
print(f'value: 0x{inv:02x}')

187
value: 0xbb


In [38]:
A=[
    [1,0,0,0,1,1,1,1],
    [1,1,0,0,0,1,1,1],
    [1,1,1,0,0,0,1,1],
    [1,1,1,1,0,0,0,1],
    [1,1,1,1,1,0,0,0],
    [0,1,1,1,1,1,0,0],
    [0,0,1,1,1,1,1,0],
    [0,0,0,1,1,1,1,1]
]

print(f"{inv:08b}")
B=[int(b) for b in f"{inv:08b}"]

print("B =", B)
B=B[::-1]
print("Reversed B =", B)

C=[0]*8
X=[0]*8
D=[1,1,0,0,0,1,1,0]

10111011
B = [1, 0, 1, 1, 1, 0, 1, 1]
Reversed B = [1, 1, 0, 1, 1, 1, 0, 1]


In [39]:
for i in range(8):
    val=0
    for j in range(8):
        val ^= A[i][j] & B[j]
    C[i]=val
print("C =", C)

C = [0, 0, 1, 0, 0, 0, 1, 0]


In [43]:
X=[C[i]^D[i] for i in range(8)]
print("X =", X)
X=X[::-1]

X = [1, 1, 1, 0, 0, 1, 0, 0]


In [ ]:

print("Reversed X =", X)

output_string=''.join(str(i) for i in X)
output_value=int(output_string,2)
print(output_value)
print(f'Output value: 0x{output_value:02x}')

hex_value=hex(output_value)
print(f'Hex value: {hex_value[2:]}')
z=hex_value[2:]
print(f'Final Output:{z}')

# print(f"{inv:08b}")
# B=[int(b) for b in f"{inv:08b}"]
print(f"{int(z, 16):08b}")


Reversed X = [0, 0, 1, 0, 0, 1, 1, 1]
39
Output value: 0x27
Hex value: 27
Final Output:27
00100111


In [78]:
val=input()
print(val)
print(type(val))
hex_val=hex(int(val,16))
print(type(hex_val))
inv=gf_inv(int(hex_val, 16))


print(inv)
print(f"{inv:02x}")

3d
<class 'str'>
<class 'str'>
187
bb


In [83]:
import numpy as np
val=input()
inv=gf_inv(int(val,16))

print(f"{inv:02x}")


bb


In [88]:
A=[
    [1,0,0,0,1,1,1,1],
    [1,1,0,0,0,1,1,1],
    [1,1,1,0,0,0,1,1],
    [1,1,1,1,0,0,0,1],
    [1,1,1,1,1,0,0,0],
    [0,1,1,1,1,1,0,0],
    [0,0,1,1,1,1,1,0],
    [0,0,0,1,1,1,1,1]
]

B=[int(i) for i in f"{inv:08b}"]

print("B =", B)
B=B[::-1]
print("Reversed B =", B)


B = [1, 0, 1, 1, 1, 0, 1, 1]
Reversed B = [1, 1, 0, 1, 1, 1, 0, 1]


In [89]:
D=[1,1,0,0,0,1,1,0]

C=[0]*8
X=[0]*8

for i in range(8):
    val =0
    for j in range(8):
        val^=A[i][j]&B[j]
    C[i]=val
print("C =", C)

C = [0, 0, 1, 0, 0, 0, 1, 0]


In [95]:
for i in range(8):
    X[i]=C[i]^D[i]


print("X =", X)
X=X[::-1]
print("Reversed X =", X)

out_str=''.join(str(i) for i in X)
print("Output string =", out_str)

output_value=int(out_str,2)
print("Output value =", output_value)

hex_value=hex(output_value)
print(f'Hex value: 0x{hex_value[2:]}')


X = [1, 1, 1, 0, 0, 1, 0, 0]
Reversed X = [0, 0, 1, 0, 0, 1, 1, 1]
Output string = 00100111
Output value = 39
Hex value: 0x27


RSA practices

In [96]:
import math

In [106]:
p,q=1709,1109
n=p*q
phi=(p-1)*(q-1)

e=3 # 1<e<phi(n) and gcd(e,phi(n))=1

if math.gcd(e,phi)!=1:
    raise ValueError("e and phi(n) are not coprime")


# Compute (a^-1 mod phi) using the Extended Euclidean Algorithm
def mod_inverse(a: int, phi: int) -> int:
    a = a % phi
    if a == 0:
        raise ValueError("Inverse does not exist for 0")

    t0, t1 = 0, 1
    r0, r1 = phi, a

    while r1 != 0:
        q = r0 // r1
        r0, r1 = r1, r0 - q * r1
        t0, t1 = t1, t0 - q * t1

    if r0 != 1:
        raise ValueError(f"No modular inverse exists for {a} mod {phi}")

    # Make sure result is positive
    return t0 % phi


d=mod_inverse(e,phi)

print(e*d%phi)


x=30

y=pow(x,e,n)
print(f"Encrypted value: {y}")

decrypted_x=pow(y,d,n)
print(f"Decrypted value: {decrypted_x}")


pt="shojib kmn aso?"

en=[pow(ord(i),e,n) for i in pt] #pt[i]^e mod n

print("Encrypted:", en)

de=''.join(chr(pow(i,d,n)) for i in en) #en[i]^d mod n
print("Decrypted:", de)








1
Encrypted value: 27000
Decrypted value: 30
Encrypted: [1520875, 1124864, 1367631, 1191016, 1157625, 941192, 32768, 1225043, 1295029, 1331000, 32768, 912673, 1520875, 1367631, 250047]
Decrypted: shojib kmn aso?


In [118]:
#rsa
from math import gcd

p,q=1709,1109
n=p*q
phi=(p-1)*(q-1)
e=3

if gcd(e,phi)!=1:
    raise ValueError("e and phi(n) are not coprime")

d=mod_inverse(e,phi)
print(e*d%phi)

x=8908
en=pow(x,e,n)
de=pow(en,d,n)

print(f"Encrypted value: {en}")
print(f"Decrypted value: {de}")

pt="shojib"

en=[pow(ord(ch),e,n) for ch in pt]

print("Encrypted:", en)

de=''.join(chr(pow(i,d,n)) for i in en)
print("Decrypted:", de)






1
Encrypted value: 166428
Decrypted value: 8908
Encrypted: [1520875, 1124864, 1367631, 1191016, 1157625, 941192]
Decrypted: shojib


In [ ]:
#defel hill man

